This is based off of this Project:

1) provides the forward model of the imaging process.  Namely given a numerical phantom (“image”) of say the head), takes user provided field maps for the B0 magnet, and the gradients and RF coil and generates what the kspace of that object is expected to be.
2) Show the conventional FFT reconstruction of this modeled data and compare it to the ground truth object with some set of metrics (of distortion etc.)
3) Do an iterative reconstruction of the ground-truth object using the forward model to give a "best case” look at how this configuration of fields would image.  Note that if the fields render an ill-posed inverse problem, there will be residual aliasing and noise amplification.  Provide metrics of these.

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

N_spins = 81  # Number of spins in each direction
# Create the phantom
phantom = np.ones((N_spins, N_spins), dtype=np.complex128)  # All spins start in equilibrium state

gamma = 42.58e6  # Gyromagnetic ratio for hydrogen in Hz/T
G = 1e-3  # Gradient strength in T/m

# Load the CSV B0 data into a NumPy array
B0 = np.genfromtxt('2Dslice.csv', delimiter=',')

# Calculate the Larmor frequency
omega0 = B0 * gamma

# Calculate the magnetic field strength
B = G * omega0
